RAG-based AI Log Analysis with LangChain

Overview

This document outlines the implementation of a Retrieval-Augmented Generation (RAG)-based AI system for log analysis. The system utilizes LangChain, FAISS, and OpenAI to enable intelligent querying of log files, providing contextual and insightful responses to user queries.

Implementation Steps

## 1. Import Required Libraries

In [ ]:
import os
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import faiss
from langchain.chains import VectorDBQA
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI
import gradio as gr

## 2. Log Monitor Module

This module uses the watchdog library to monitor log file updates and trigger processing for new log entries.

In [ ]:
class LogMonitor(FileSystemEventHandler):
    def __init__(self, log_file_path, on_update_callback):
        self.log_file_path = log_file_path
        self.on_update_callback = on_update_callback

    def on_modified(self, event):
        if event.src_path == self.log_file_path:
            with open(self.log_file_path, 'r') as f:
                lines = f.readlines()
            self.on_update_callback(lines)


## 3. Preprocessor Module

This module prepares raw log entries for vectorization by performing basic text cleaning and normalization.

In [ ]:
def preprocess_logs(log_lines):
    processed_logs = []
    for line in log_lines:
        processed_logs.append(line.strip().lower())
    return processed_logs

## 4. Vectorizer and FAISS Integration with LangChain

The integration of FAISS and LangChain enables efficient storage and retrieval of log entries based on semantic similarity.

In [ ]:
class LangChainFAISSHandler:
    def __init__(self):
        self.embedding = OpenAIEmbeddings()
        self.vector_store = FAISS(embedding_function=self.embedding)
        self.system_context = "You are a helpful and intelligent log analyzer. Answer user queries based on the provided log data."

    def add_logs(self, logs):
        for log in logs:
            self.vector_store.add_texts([log])

    def query_logs(self, message, history):
        relevant_system_context = self.system_context
        messages = (
            [{"role": "system", "content": relevant_system_context}] + history + [{"role": "user", "content": message}]
        )

        qa_chain = VectorDBQA(llm=OpenAI(), vectorstore=self.vector_store)
        response = qa_chain.run(messages)
        return response

## 5. Integration Workflow

This workflow ties together log monitoring, preprocessing, vectorization, and querying into a seamless pipeline.

In [ ]:
def process_logs_and_query(log_file_path, message, history):
    # Step 1: Monitor Logs
    if not hasattr(process_logs_and_query, "faiss_handler"):
        process_logs_and_query.faiss_handler = LangChainFAISSHandler()

    def on_log_update(new_logs):
        processed_logs = preprocess_logs(new_logs)
        process_logs_and_query.faiss_handler.add_logs(processed_logs)

    log_monitor = LogMonitor(log_file_path, on_log_update)
    observer = Observer()
    observer.schedule(log_monitor, path=os.path.dirname(log_file_path), recursive=False)
    observer.start()

    # Step 2: Query Logs
    return process_logs_and_query.faiss_handler.query_logs(message, history)


# 6. Entry Point for Chat Integration

To enable user interaction through a UI like Gradio or other communication channels (e.g., Webex bot), the following entry point is provided:

In [ ]:
def chat(message, history):
    log_file_path = '/path/to/logfile.log'  # Path to your log file
    history = history or []
    response = process_logs_and_query(log_file_path, message, history)
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})
    return response

# Gradio UI
chat_ui = gr.ChatInterface(
    fn=chat,
    type="messages"
)
chat_ui.launch()